### Importing Dependencies

In [2]:
import pandas as pd
from sqlalchemy import create_engine
import requests
import time
from pprint import pprint
import numpy as np
import json

# Import API key
from config import api_key

### Extract CSV into DataFrames

In [50]:
#Food Information dataset from Kaggle
csv_file = "kaggle_food.csv"
nutrients_df = pd.read_csv(csv_file)
nutrients_df.head()

,Category,Description,Nutrient Data Bank Number,Data.Alpha Carotene,Data.Ash,Data.Beta Carotene,Data.Beta Cryptoxanthin,Data.Carbohydrate,Data.Cholesterol,Data.Choline,...,Data.Major Minerals.Potassium,Data.Major Minerals.Sodium,Data.Major Minerals.Zinc,Data.Vitamins.Vitamin A - IU,Data.Vitamins.Vitamin A - RAE,Data.Vitamins.Vitamin B12,Data.Vitamins.Vitamin B6,Data.Vitamins.Vitamin C,Data.Vitamins.Vitamin E,Data.Vitamins.Vitamin K
0,BUTTER,"BUTTER,WITH SALT",1001,0,2.11,158,0,0.06,215,19,...,24,576,0.09,2499,684,0.17,0.003,0.0,2.32,7.0
1,BUTTER,"BUTTER,WHIPPED,WITH SALT",1002,0,2.11,158,0,0.06,219,19,...,26,827,0.05,2499,684,0.13,0.003,0.0,2.32,7.0
2,BUTTER OIL,"BUTTER OIL,ANHYDROUS",1003,0,0.00,193,0,0.00,256,22,...,5,2,0.01,3069,840,0.01,0.001,0.0,2.80,8.6
3,CHEESE,"CHEESE,BLUE",1004,0,5.11,74,0,2.34,75,15,...,256,1395,2.66,763,198,1.22,0.166,0.0,0.25,2.4
4,CHEESE,"CHEESE,BRICK",1005,0,3.18,76,0,2.79,94,15,...,136,560,2.60,1080,292,1.26,0.065,0.0,0.26,2.5


In [3]:
#Recipes by Ingredients dataset from Kaggle
csv_file = "master_recipe.csv"
recipes_df = pd.read_csv(csv_file, index_col=False)
recipes_df.head()

,label,achiote paste,achiote powder,acini di pepe,acorn squash,active dry yeast,adobo sauce,adobo seasoning,adzuki beans,agave nectar,...,yellow rock sugar,yellow squash,yellow summer squash,yellow tomato,yellowfin tuna,yogurt cheese,yucca root,yukon gold potatoes,yuzu juice,zucchini blossoms
0,Infineon Raceway Baked Beans,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Sour Cream Noodle Bake,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Middle-Eastern Eggplant Rounds,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Saffron Jewel Rice,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Thai Sweet and Sour Wings,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Transform Nutrients DataFrame

In [51]:
#retaining only the columns necessary for the database
lean_nutrient = nutrients_df.drop(columns= [
 'Data.Alpha Carotene',
 'Data.Ash',
 'Data.Beta Carotene',
 'Data.Beta Cryptoxanthin',
 'Data.Choline',
 'Data.Lutein and Zeaxanthin',
 'Data.Lycopene',
 'Data.Manganese',
 'Data.Niacin',
 'Data.Pantothenic Acid',
 'Data.Refuse Percentage',
 'Data.Retinol',
 'Data.Riboflavin',
 'Data.Selenium',
 'Data.Thiamin',
 'Data.Water',
 'Data.Fat.Total Lipid',
 'Data.Household Weights.1st Household Weight',
 'Data.Household Weights.1st Household Weight Description',
 'Data.Household Weights.2nd Household Weight',
 'Data.Household Weights.2nd Household Weight Description',
 'Data.Major Minerals.Calcium',
 'Data.Major Minerals.Copper',
 'Data.Major Minerals.Iron',
 'Data.Major Minerals.Magnesium',
 'Data.Major Minerals.Phosphorus',
 'Data.Major Minerals.Potassium',
 'Data.Major Minerals.Zinc',
 'Data.Vitamins.Vitamin A - IU',
 'Data.Vitamins.Vitamin A - RAE',
 'Data.Vitamins.Vitamin B12',
 'Data.Vitamins.Vitamin B6',
 'Data.Vitamins.Vitamin C',
 'Data.Vitamins.Vitamin E',
 'Data.Vitamins.Vitamin K'])

lean_nutrient.head()

,Category,Description,Nutrient Data Bank Number,Data.Carbohydrate,Data.Cholesterol,Data.Fiber,Data.Kilocalories,Data.Protein,Data.Sugar Total,Data.Fat.Monosaturated Fat,Data.Fat.Polysaturated Fat,Data.Fat.Saturated Fat,Data.Major Minerals.Sodium
0,BUTTER,"BUTTER,WITH SALT",1001,0.06,215,0.0,717,0.85,0.06,21.021,3.043,51.368,576
1,BUTTER,"BUTTER,WHIPPED,WITH SALT",1002,0.06,219,0.0,717,0.85,0.06,23.426,3.012,50.489,827
2,BUTTER OIL,"BUTTER OIL,ANHYDROUS",1003,0.00,256,0.0,876,0.28,0.00,28.732,3.694,61.924,2
3,CHEESE,"CHEESE,BLUE",1004,2.34,75,0.0,353,21.40,0.50,7.778,0.800,18.669,1395
4,CHEESE,"CHEESE,BRICK",1005,2.79,94,0.0,371,23.24,0.51,8.598,0.784,18.764,560


In [61]:
rename_nutrients = lean_nutrient.rename(columns={"Category" : "ingredient",
                                                "Description" : "food_description",
                                                "Nutrient Data Bank Number" : "ingredient_id",
                                                "Data.Carbohydrate" : "carbs",
                                                 "Data.Protein" : "protein",
                                                "Data.Cholesterol" : "cholesterol",
                                                "Data.Fiber" : "fiber",
                                                "Data.Kilocalories" : "calories",
                                                "Data.Sugar Total" : "sugar",
                                                "Data.Fat.Monosaturated Fat": "monosaturated_fat",
                                                "Data.Fat.Polysaturated Fat" : "polysaturated_fat",
                                                "Data.Fat.Saturated Fat" : "saturated_fat",
                                                "Data.Major Minerals.Sodium" : "sodium"})

rename_nutrients

,ingredient,food_description,ingredient_id,carbs,cholesterol,fiber,calories,protein,sugar,monosaturated_fat,polysaturated_fat,saturated_fat,sodium
0,BUTTER,"BUTTER,WITH SALT",1001,0.06,215,0.0,717,0.85,0.06,21.021,3.043,51.368,576
1,BUTTER,"BUTTER,WHIPPED,WITH SALT",1002,0.06,219,0.0,717,0.85,0.06,23.426,3.012,50.489,827
2,BUTTER OIL,"BUTTER OIL,ANHYDROUS",1003,0.00,256,0.0,876,0.28,0.00,28.732,3.694,61.924,2
3,CHEESE,"CHEESE,BLUE",1004,2.34,75,0.0,353,21.40,0.50,7.778,0.800,18.669,1395
4,CHEESE,"CHEESE,BRICK",1005,2.79,94,0.0,371,23.24,0.51,8.598,0.784,18.764,560
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7408,FROG LEGS,"FROG LEGS,RAW",80200,0.00,50,0.0,73,16.40,0.00,0.053,0.102,0.076,58
7409,MACKEREL,"MACKEREL,SALTED",83110,0.00,95,0.0,305,18.50,0.00,8.320,6.210,7.148,4450
7410,SCALLOP,"SCALLOP,(BAY&SEA),CKD,STMD",90240,0.00,53,0.0,112,23.20,0.00,0.068,0.481,0.146,265
7411,SNAIL,"SNAIL,RAW",90560,2.00,50,0.0,90,16.10,0.00,0.259,0.252,0.361,70


In [132]:
#checking for missing values
null_check = rename_nutrients.isnull().sum().sum()

null_check

0

In [62]:
#final dataset to load into SQL
rename_nutrients.to_csv("output/kaggle_nutrients.csv",index=False)


### Transform Recipes DataFrame

In [4]:
#transposing the dataframe
transposed_recipes =recipes_df.set_index('label').T

transposed_recipes

label,Infineon Raceway Baked Beans,Sour Cream Noodle Bake,Middle-Eastern Eggplant Rounds,Saffron Jewel Rice,Thai Sweet and Sour Wings,Balsamic-Marinated Steak and Unstuffed Mushrooms,Baked Chicken Saltimbocca,Zucchini Carpaccio,Happy Holly's Banana Cream Pie,5 Taste Lime Slime Garnished with Mad Rocks,...,"Gianduja Caramel ""Mud""","Strawberry, Fig, and Goat Cheese Tarts",Eggnog-Chocolate Cake Roll,Grilled Honey-Orange Figs with Mascarpone and Pistachios,Pork Shoulder Rajas with Quick Kimchi in Lettuce Leaves with Kojuchang Sauce,Spicy Agave Wave,Apple Chimichangas,Summer Corn Salad,Zucchini Stuffed Tomatoes,Crabby Bisque
achiote paste,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
achiote powder,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
acini di pepe,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
acorn squash,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
active dry yeast,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
yogurt cheese,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
yucca root,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
yukon gold potatoes,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
yuzu juice,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
#reset the index
transposed_recipes.reset_index(inplace=True)

transposed_recipes

label,index,Infineon Raceway Baked Beans,Sour Cream Noodle Bake,Middle-Eastern Eggplant Rounds,Saffron Jewel Rice,Thai Sweet and Sour Wings,Balsamic-Marinated Steak and Unstuffed Mushrooms,Baked Chicken Saltimbocca,Zucchini Carpaccio,Happy Holly's Banana Cream Pie,...,"Gianduja Caramel ""Mud""","Strawberry, Fig, and Goat Cheese Tarts",Eggnog-Chocolate Cake Roll,Grilled Honey-Orange Figs with Mascarpone and Pistachios,Pork Shoulder Rajas with Quick Kimchi in Lettuce Leaves with Kojuchang Sauce,Spicy Agave Wave,Apple Chimichangas,Summer Corn Salad,Zucchini Stuffed Tomatoes,Crabby Bisque
0,achiote paste,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,achiote powder,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,acini di pepe,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,acorn squash,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,active dry yeast,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2919,yogurt cheese,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2920,yucca root,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2921,yukon gold potatoes,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2922,yuzu juice,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
rename_recipe = transposed_recipes.rename(columns={"index" : "Ingredient"})

rename_recipe

label,Ingredient,Infineon Raceway Baked Beans,Sour Cream Noodle Bake,Middle-Eastern Eggplant Rounds,Saffron Jewel Rice,Thai Sweet and Sour Wings,Balsamic-Marinated Steak and Unstuffed Mushrooms,Baked Chicken Saltimbocca,Zucchini Carpaccio,Happy Holly's Banana Cream Pie,...,"Gianduja Caramel ""Mud""","Strawberry, Fig, and Goat Cheese Tarts",Eggnog-Chocolate Cake Roll,Grilled Honey-Orange Figs with Mascarpone and Pistachios,Pork Shoulder Rajas with Quick Kimchi in Lettuce Leaves with Kojuchang Sauce,Spicy Agave Wave,Apple Chimichangas,Summer Corn Salad,Zucchini Stuffed Tomatoes,Crabby Bisque
0,achiote paste,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,achiote powder,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,acini di pepe,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,acorn squash,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,active dry yeast,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2919,yogurt cheese,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2920,yucca root,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2921,yukon gold potatoes,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2922,yuzu juice,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [10]:
#checking for missing values
null_check = rename_recipe.isnull().sum().sum()

null_check

0

In [11]:
#final dataset to load into SQL
rename_recipe.to_csv("output/kaggle_recipes.csv",index=False)

In [22]:
master_ingredient = pd.DataFrame(rename_recipe.loc[:, 'Ingredient'])

master_ingredient

,Ingredient
0,achiote paste
1,achiote powder
2,acini di pepe
3,acorn squash
4,active dry yeast
...,...
2919,yogurt cheese
2920,yucca root
2921,yukon gold potatoes
2922,yuzu juice


### Attempt to Call on the Chomp API

In [ ]:
### Note: these items were not found in Chomp. Output below shows the outcome

In [12]:
#setting up additional columns to hold information
rename_recipe['Protein'] = ""
rename_recipe['Fat'] = ""
rename_recipe['Carbs'] = ""

rename_recipe.head()

label,Ingredient,Infineon Raceway Baked Beans,Sour Cream Noodle Bake,Middle-Eastern Eggplant Rounds,Saffron Jewel Rice,Thai Sweet and Sour Wings,Balsamic-Marinated Steak and Unstuffed Mushrooms,Baked Chicken Saltimbocca,Zucchini Carpaccio,Happy Holly's Banana Cream Pie,...,Grilled Honey-Orange Figs with Mascarpone and Pistachios,Pork Shoulder Rajas with Quick Kimchi in Lettuce Leaves with Kojuchang Sauce,Spicy Agave Wave,Apple Chimichangas,Summer Corn Salad,Zucchini Stuffed Tomatoes,Crabby Bisque,Protein,Fat,Carbs
0,achiote paste,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,,,
1,achiote powder,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,,,
2,acini di pepe,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,,,
3,acorn squash,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,,,
4,active dry yeast,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,,,


In [66]:
#using iterrows to iterate throguh pandas dataframe
for index, row in rename_recipe.iterrows():
    #obtaining the ingredient name
    ingredient = row["Ingredient"]
        
    #making the url for requests 
    print(f"Retrieving Results for {ingredient}.")
    response = requests.get(f'https://chompthis.com/api/v2/food/ingredient/search.php?api_key=169jhmSznT0112Pmm&find={ingredient}')
    time.sleep(1)
                            
    #converting response into json
    response=response.json()
    pprint(response)
    #try to grab the following information, if available
    try:
        rename_recipe.loc[index, 'Protein'] = response["calorie_conversion_factor"][0]["protein_value"]
        rename_recipe.loc[index, 'Fat'] = response["protein_value"]
        rename_recipe.loc[index, 'Carbs'] = response["name"][0]["Carbohydrate, by difference"][0]["per 100 g"]
        
           
    #including a condition for ingredients that are not found in Chomp this
    except (KeyError, IndexError):
        print(f"Ingredient not found. Skipping...")

Retrieving Results for achiote paste.
{'error': '404 Not Found: No food items were found. Please try again in a '
          'minute or contact us. '
          'https://chompthis.com/api/ticket-new.php?ref=404'}
Ingredient not found. Skipping...
Retrieving Results for achiote powder.
{'error': '404 Not Found: No food items were found. Please try again in a '
          'minute or contact us. '
          'https://chompthis.com/api/ticket-new.php?ref=404'}
Ingredient not found. Skipping...
Retrieving Results for acini di pepe.
{'error': '404 Not Found: No food items were found. Please try again in a '
          'minute or contact us. '
          'https://chompthis.com/api/ticket-new.php?ref=404'}
Ingredient not found. Skipping...
Retrieving Results for acorn squash.
{'items': [{'calorie_conversion_factor': {'carbohydrate_value': None,
                                          'fat_value': None,
                                          'protein_value': None},
            'categories': ['Ve

{'error': '404 Not Found: No food items were found. Please try again in a '
          'minute or contact us. '
          'https://chompthis.com/api/ticket-new.php?ref=404'}
Ingredient not found. Skipping...
Retrieving Results for adobo sauce.
{'error': '404 Not Found: No food items were found. Please try again in a '
          'minute or contact us. '
          'https://chompthis.com/api/ticket-new.php?ref=404'}
Ingredient not found. Skipping...
Retrieving Results for adobo seasoning.
{'error': '404 Not Found: No food items were found. Please try again in a '
          'minute or contact us. '
          'https://chompthis.com/api/ticket-new.php?ref=404'}
Ingredient not found. Skipping...
Retrieving Results for adzuki beans.
{'items': [{'calorie_conversion_factor': {'carbohydrate_value': None,
                                          'fat_value': None,
                                          'protein_value': None},
            'categories': ['Legumes and Legume Products'],
          

{'error': '404 Not Found: No food items were found. Please try again in a '
          'minute or contact us. '
          'https://chompthis.com/api/ticket-new.php?ref=404'}
Ingredient not found. Skipping...
Retrieving Results for agave tequila.
{'error': '404 Not Found: No food items were found. Please try again in a '
          'minute or contact us. '
          'https://chompthis.com/api/ticket-new.php?ref=404'}
Ingredient not found. Skipping...
Retrieving Results for aged balsamic vinegar.
{'error': '404 Not Found: No food items were found. Please try again in a '
          'minute or contact us. '
          'https://chompthis.com/api/ticket-new.php?ref=404'}
Ingredient not found. Skipping...
Retrieving Results for aged cheddar cheese.
{'error': '404 Not Found: No food items were found. Please try again in a '
          'minute or contact us. '
          'https://chompthis.com/api/ticket-new.php?ref=404'}
Ingredient not found. Skipping...
Retrieving Results for aged gouda.
{'error':

{'items': [{'calorie_conversion_factor': {'carbohydrate_value': None,
                                          'fat_value': None,
                                          'protein_value': None},
            'categories': ['Soups, Sauces, and Gravies'],
            'common_name': None,
            'components': [],
            'footnote': None,
            'name': 'Sauce, alfredo mix, dry',
            'nutrients': [{'data_points': 3,
                           'description': 'Manufacturer supplied(industry or '
                                          'trade association), Analytical '
                                          'data, incomplete documentation',
                           'footnote': '',
                           'max': 0,
                           'measurement_unit': 'G',
                           'median': 0,
                           'min': 0,
                           'name': 'Water',
                           'per_100g': 2.05,
                           'ran

{'error': '404 Not Found: No food items were found. Please try again in a '
          'minute or contact us. '
          'https://chompthis.com/api/ticket-new.php?ref=404'}
Ingredient not found. Skipping...
Retrieving Results for allspice berries.
{'error': '404 Not Found: No food items were found. Please try again in a '
          'minute or contact us. '
          'https://chompthis.com/api/ticket-new.php?ref=404'}
Ingredient not found. Skipping...
Retrieving Results for almond butter.
{'items': [{'calorie_conversion_factor': {'carbohydrate_value': None,
                                          'fat_value': None,
                                          'protein_value': None},
            'categories': [],
            'common_name': None,
            'components': [],
            'footnote': None,
            'name': 'Almond butter',
            'nutrients': [],
            'portions': [{'data_points': None,
                          'description': '1 tablespoon',
                 

                           'min': 8.964,
                           'name': '18:2',
                           'per_100g': 13.605,
                           'rank': 13100},
                          {'data_points': 8,
                           'description': 'Analytical',
                           'footnote': '',
                           'max': 0.007,
                           'measurement_unit': 'G',
                           'median': 0,
                           'min': 0.007,
                           'name': '18:3',
                           'per_100g': 0.007,
                           'rank': 13900},
                          {'data_points': 8,
                           'description': 'Analytical',
                           'footnote': '',
                           'max': 0,
                           'measurement_unit': 'G',
                           'median': 0,
                           'min': 0,
                           'name': '20:2 n-6 c,c',
               

                                          'or similar food; Concentration '
                                          'adjustment; No adjustment; '
                                          'Retention factors not used',
                           'footnote': '',
                           'max': 0,
                           'measurement_unit': 'UG',
                           'median': 0,
                           'min': 0,
                           'name': 'Carotene, beta',
                           'per_100g': 1,
                           'rank': 7440},
                          {'data_points': 0,
                           'description': 'Based on another form of the food '
                                          'or similar food; Concentration '
                                          'adjustment; No adjustment; '
                                          'Retention factors not used',
                           'footnote': '',
                           'max': 0,
         

{'error': '404 Not Found: No food items were found. Please try again in a '
          'minute or contact us. '
          'https://chompthis.com/api/ticket-new.php?ref=404'}
Ingredient not found. Skipping...
Retrieving Results for almond filling.
{'error': '404 Not Found: No food items were found. Please try again in a '
          'minute or contact us. '
          'https://chompthis.com/api/ticket-new.php?ref=404'}
Ingredient not found. Skipping...
Retrieving Results for almond flour.
{'error': '404 Not Found: No food items were found. Please try again in a '
          'minute or contact us. '
          'https://chompthis.com/api/ticket-new.php?ref=404'}
Ingredient not found. Skipping...
Retrieving Results for almond liqueur.
{'error': '404 Not Found: No food items were found. Please try again in a '
          'minute or contact us. '
          'https://chompthis.com/api/ticket-new.php?ref=404'}
Ingredient not found. Skipping...
Retrieving Results for almond meal.


KeyboardInterrupt: 

In [ ]:
pprint(response)

In [40]:
null_check = rename_recipe.isnull().sum().sum()

null_check

0

In [67]:
master_ingred_df = rename_recipe[['Ingredient', 'Protein', 'Fat', 'Carbs']]

master_ingred_df.head(50)

label,Ingredient,Protein,Fat,Carbs
0,achiote paste,,,
1,achiote powder,,,
2,acini di pepe,,,
3,acorn squash,,,
4,active dry yeast,,,
5,adobo sauce,,,
6,adobo seasoning,,,
7,adzuki beans,,,
8,agave nectar,,,
9,agave tequila,,,


In [ ]:
#final dataset to load into SQL
master_ingred.to_csv("output/master_kaggle_ingred.csv",index=False)

### Create database connection

In [17]:
connection_string = "postgres:postgres@localhost:5432/kaggle_nutrition_db"
engine = create_engine(f'postgresql://{connection_string}')

In [18]:
# Confirm tables
engine.table_names()

<ipython-input-18-2d97e9123e98>:2: SADeprecationWarning: The Engine.table_names() method is deprecated and will be removed in a future release.  Please refer to Inspector.get_table_names(). (deprecated since: 1.4)
  engine.table_names()


OperationalError: (psycopg2.OperationalError) connection to server at "localhost" (::1), port 5432 failed: FATAL:  database "nutrient_db" does not exist

(Background on this error at: http://sqlalche.me/e/14/e3q8)

### Load DataFrames into database

In [53]:
#checking data types
rename_nutrients.dtypes


ingredient            object
food_description      object
ingredient_iD          int64
carbs                float64
cholesterol            int64
fiber                float64
calories               int64
protein              float64
sugar                float64
monosaturated_fat    float64
polysaturated_fat    float64
saturated_fat        float64
sodium                 int64
dtype: object

In [54]:
#import dependencies
from sqlalchemy import create_engine
import psycopg2


In [55]:
#setting up the connection to pgAdmin
rds_connection_string = "postgres:postgres@localhost:5432/kaggle_nutrition_db"

engine = create_engine(f'postgresql://{rds_connection_string}')


In [57]:
#obtaining table names
engine.table_names()

<ipython-input-57-d72bc75a6b62>:1: SADeprecationWarning: The Engine.table_names() method is deprecated and will be removed in a future release.  Please refer to Inspector.get_table_names(). (deprecated since: 1.4)
  engine.table_names()


['kaggle_nutrition']

In [63]:
#reading in the 
df = pd.read_csv ('output/kaggle_nutrients.csv')

df.head()


,ingredient,food_description,ingredient_id,carbs,cholesterol,fiber,calories,protein,sugar,monosaturated_fat,polysaturated_fat,saturated_fat,sodium
0,BUTTER,"BUTTER,WITH SALT",1001,0.06,215,0.0,717,0.85,0.06,21.021,3.043,51.368,576
1,BUTTER,"BUTTER,WHIPPED,WITH SALT",1002,0.06,219,0.0,717,0.85,0.06,23.426,3.012,50.489,827
2,BUTTER OIL,"BUTTER OIL,ANHYDROUS",1003,0.00,256,0.0,876,0.28,0.00,28.732,3.694,61.924,2
3,CHEESE,"CHEESE,BLUE",1004,2.34,75,0.0,353,21.40,0.50,7.778,0.800,18.669,1395
4,CHEESE,"CHEESE,BRICK",1005,2.79,94,0.0,371,23.24,0.51,8.598,0.784,18.764,560


In [64]:
df.to_sql(name='kaggle_nutrition', con=engine, if_exists='append', index=False)
